<a href="https://colab.research.google.com/github/111Vidya/AI-Fundametals/blob/main/AI_Jun5_Text_Generation_with_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="http://colab.research.google.com/github/dipanjanS/nlp_workshop_odsc19/blob/master/Module05%20-%20NLP%20Applications/Project08%20-%20Text%20Generation%20with%20Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Generation with Transformers

It turns out we don’t need an entire Transformer to adopt transfer learning and a fine-tunable language model for NLP tasks. We can do with just the decoder of the transformer. The decoder is a good choice because it’s a natural choice for language modeling (predicting the next word) since it’s built to mask future tokens – a valuable feature when it’s generating a translation word by word.

Here we will use the GPT-2 Model to generate text based on an input sequence of text.

![](https://i.imgur.com/z4k1IzU.png)

# Install Dependencies

In [ ]:
!pip install pytorch-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 941.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 38.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-m

# Load GPT2 Model

In [ ]:
import torch
from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

100%|██████████| 456318/456318 [00:00<00:00, 1761997.60B/s]


# Next Word Generation with GPT-2

GPT-2 is a successor of GPT, the original NLP framework by OpenAI. The full GPT-2 model has 1.5 billion parameters, which is almost 10 times the parameters of GPT. GPT-2 give State-of-the Art results as you might have surmised already (and will soon see when we get into Python).

The pre-trained model contains data from 8 million web pages collected from outbound links from Reddit.

![](https://i.imgur.com/TbnGbjX.png)

The architecture of GPT-2 is based on the very famous Transformers concept that was proposed by Google in their paper “Attention is all you need”. The Transformer provides a mechanism based on encoder-decoders to detect input-output dependencies.

At each step, the model consumes the previously generated symbols as additional input when generating the next output.

![](https://i.imgur.com/0XSSXBd.png)

Modifications in GPT-2 include:

- The model uses larger context and vocabulary size
- After the final self-attention block, an additional normalization layer is added
- Similar to a residual unit of type “building block”, layer normalization is moved to the input of each sub-block. It has batch normalization applied before weight layers, which is different from the original type “bottleneck”

In [ ]:
text = "Welcome to the open data science conference it is"
indexed_tokens = tokenizer.encode(text)
indexed_tokens

[19134, 284, 262, 1280, 1366, 3783, 4495, 340, 318]

In [ ]:
tokens_tensor = torch.tensor([indexed_tokens])
tokens_tensor

tensor([[19134,   284,   262,  1280,  1366,  3783,  4495,   340,   318]])

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()
model

100%|██████████| 548118077/548118077 [00:12<00:00, 42289980.54B/s]


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

In [ ]:
predictions.shape

torch.Size([1, 9, 50257])

In [ ]:
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
predicted_text

' Welcome to the open data science conference it is a'

'a' is the predicted word

In [ ]:
start = 'In a distant future, humanity has discovered'
indexed_tokens = tokenizer.encode(start)

for i in range(75):
  tokens_tensor = torch.tensor([indexed_tokens])
  with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]
    predicted_index = torch.argmax(predictions[0, -1, :]).item()
    indexed_tokens = indexed_tokens + [predicted_index]

In [ ]:
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
print(predicted_text)

 In a distant future, humanity has discovered a way to harness the power of the sun to create a new kind of energy.

The sun is a powerful force that can be harnessed to create new forms of life. It is the source of the energy that powers the universe.

The sun is the source of the energy that powers the universe. The sun is the source of the energy that powers the the


# Paragraph Generation with GPT-2

Refer to this [source code](https://github.com/huggingface/pytorch-transformers/blob/master/examples/run_generation.py#L106-L129) to deep dive.

- `length`: It represents the number of tokens in the generated text. If the length is None, then the number of tokens is decided by model hyperparameters
- `temperature`: This controls randomness in Boltzmann distribution. Lower temperature results in less random completions. As the temperature approaches zero, the model will become deterministic and repetitive. Higher temperature results in more random completions
- `top_k`: This parameter controls diversity. If the value of top_k is set to 1, this means that only 1 word is considered for each step (token). If top_k is set to 40, that means 40 words are considered at each step. 0 (default) is a special setting meaning no restrictions. top_k = 40 generally is a good value

In [ ]:
!git clone https://github.com/huggingface/pytorch-transformers.git

Cloning into 'pytorch-transformers'...
remote: Enumerating objects: 220093, done.
remote: Counting objects: 100% (543/543), done.
remote: Compressing objects: 100% (298/298), done.
remote: Total 220093 (delta 282), reused 377 (delta 199), pack-reused 219550
Receiving objects: 100% (220093/220093), 229.90 MiB | 22.27 MiB/s, done.


In [ ]:
!python pytorch-transformers/examples/run_generation.py \
    --model_type=gpt2 \
    --length=500 \
    --model_name_or_path=gpt2 \